#MILESTONE 4#
Ali Wehbi - Dina Younes - Reeda Al Saintbai - Shaza Fakih - Youssef Jaafar

In [ ]:

from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
#df = pd.read_csv("/content/drive/iseardataset.csv")       #importing the ISEAR dataset into a dataframe
df = pd.read_csv("/content/iseardataset.csv")   
print(df.head())


     label                                               text Unnamed: 2
0      joy  On days when I feel close to my partner and ot...        NaN
1     fear  Every time I imagine that someone I love or I ...        NaN
2    anger  When I had been obviously unjustly treated and...        NaN
3  sadness  When I think about the short time that we live...        NaN
4  disgust  At a gathering I found myself involuntarily si...        NaN


In [ ]:
#Removing last column
df = df[['label','text']] 
print(df.head())

     label                                               text
0      joy  On days when I feel close to my partner and ot...
1     fear  Every time I imagine that someone I love or I ...
2    anger  When I had been obviously unjustly treated and...
3  sadness  When I think about the short time that we live...
4  disgust  At a gathering I found myself involuntarily si...


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7516 entries, 0 to 7515
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   7516 non-null   object
 1   text    7516 non-null   object
dtypes: object(2)
memory usage: 117.6+ KB





#Preprocessing of data#

In [ ]:
#importing necessary libraries for preprocessing
import numpy as np
import itertools
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')

stop_words = stopwords.words('english')    # stop words do not contribute in giving a meaning to a sentence, hence they can be ignored

def DataPreprocess(input_text):
  """
  Function that takes text input as string, converts text into lowercase, removes non alphabetic words, tokenizes the text, 
  reduces each word to its root
  """
  input_text = input_text.lower()                   #converting to lower case
  input_text = re.sub(r'[^a-z]', ' ', input_text)   #keeping alphabetic words only
  text_tokens = word_tokenize(input_text)           #creating tokens of the text

  #print("Tokens:", text_tokens)
  #print('\n')

  porter_stemmer = PorterStemmer()                                     #initializing a stemmer
  porter_stemmed = [porter_stemmer.stem(word) for word in text_tokens] #stemming words, i.e. reducing a word to its root. 
  input_text = ' '.join(porter_stemmed)                                #after preprocessing, create full text back
  #print("After preprocessing, before stopwords:")
  #print(input_text)
  #print('\n')
  # remove stopwords
  input_text = ' '.join([word for word in input_text.split() if word not in stop_words])
  #print("After stopwords:")
  #print('\n')
  return input_text

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
df['text'] = df['text'].apply(DataPreprocess)    #Preprocessing the input data
df.head() 
print(df.head())

     label                                               text
0      joy  day feel close partner friend feel peac also e...
1     fear  everi time imagin someon love could contact se...
2    anger          obvious unjustli treat possibl elucid thi
3  sadness  think short time live relat period life think ...
4  disgust  gather found involuntarili sit next two peopl ...


#Feature Engineering & Model Building# 

First:
Using CountVectorizer

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import classification_report
X = df.text.to_numpy()        #converting the pandas series to a numpy array
#X = X.reshape(-1,1)
y = df.label.values           #returning a numpy representation of the dataframe data.label
#y = y.reshape(-1,1)
#print(X)
#print(y)
#print(X.shape)
#print(y.shape)

#Trying Bag of Words
CVect = CountVectorizer(max_features=2000)
X_CVect = CVect.fit_transform(X).toarray()
#Splitting the data into a training set and a testing set with 80-20% proportions and a random state = 52
X_train, X_test, y_train, y_test = train_test_split(X_CVect, y, test_size=0.20, random_state=52)



In [ ]:
#SVM model + CountVectorizer
model1 = svm.SVC(kernel='linear')                              #initializing an SVM model
model1.fit(X_train, y_train)                               #training the model on the training set
emotion_predicted = model1.predict(X_test)                 #measuring the quality of the model's prediction on the test set
print(classification_report(y_test, emotion_predicted))      #displaying the precision, recall, F1, and support scores for the model.
                                              

              precision    recall  f1-score   support

       anger       0.35      0.50      0.41       193
     disgust       0.53      0.48      0.50       215
        fear       0.64      0.67      0.65       208
       guilt       0.45      0.39      0.42       214
         joy       0.67      0.67      0.67       205
     sadness       0.60      0.57      0.59       228
       shame       0.50      0.44      0.47       241

    accuracy                           0.53      1504
   macro avg       0.54      0.53      0.53      1504
weighted avg       0.54      0.53      0.53      1504



Second: Using TF-IDF

In [ ]:
#Splitting the data into a training set and a testing set with 80-20% proportions and a random state = 52
X_train2, X_test2, y_train2, y_test2 = train_test_split(X, y, test_size=0.20, random_state=62)
T_vectorizer = TfidfVectorizer(min_df=2, max_df=0.9)    #instantiating the vectorizer object
vect_train = T_vectorizer.fit_transform(X_train2)        #transforming train and test data into a matrix of TF-IDF features
vect_test = T_vectorizer.transform(X_test2)            #in order to transform text into a significant representation of numbers
#print(vect_train)
#print(vect_test)



In [ ]:
#SVM + TF-IDF
model2 = svm.SVC(kernel='linear')                              #initializing an SVM model
model2.fit(vect_train, y_train2)                               #training the model on the training set
emotion_predicted2 = model2.predict(vect_test)                 #measuring the quality of the model's prediction on the test set
print(classification_report(y_test2, emotion_predicted2))      #displaying the precision, recall, F1, and support scores for the model.
                                              

              precision    recall  f1-score   support

       anger       0.46      0.58      0.51       222
     disgust       0.58      0.62      0.60       205
        fear       0.73      0.64      0.68       211
       guilt       0.48      0.44      0.46       216
         joy       0.70      0.74      0.72       213
     sadness       0.69      0.60      0.64       222
       shame       0.49      0.47      0.48       215

    accuracy                           0.58      1504
   macro avg       0.59      0.58      0.58      1504
weighted avg       0.59      0.58      0.58      1504



In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix

c_matrix = confusion_matrix(emotion_predicted2, y_test2)          #using a confusion matrix to evaluate the model
print(c_matrix)                                                 #this is useful in model management and monitoring and used to evaluate the perfomance of our classification model  
print("Accuracy: ",accuracy_score(y_test2, emotion_predicted2))   #printing the accuracy of the model after predicting on the test set


[[128  21  18  41  12  20  36]
 [ 23 127  20  10   6  13  19]
 [  7   9 136   7   6  10  12]
 [ 28  11   9  96  10  14  32]
 [  7  10  11   9 157  20  10]
 [  9   5  10  20  10 133   6]
 [ 20  22   7  33  12  12 100]]
Accuracy:  0.5831117021276596


In [ ]:
def predict(model, T_vectorizer, text):
  """
  function that takes as input a model (in our case svm), a vectorizer (tfid vectorizer in our case), and input text as string
  returns the detected emotion in the text
  """
  text = DataPreprocess(text)                      #preprocessing the input text: converting to lower case, removing non-alphabetic words, tokenizing the text, and stemming words
  text_vectorized = T_vectorizer.transform([text]) #using the tfidf vectorizer on the input text
  return model.predict(text_vectorized)[0]         #running the model on the vectorizer




In [ ]:
text1 = "Today i went to the mall with my friends to celebrate my friend's birthday, it was great!"
print("Emotion predicted: ",predict(model2,T_vectorizer,text1))

Emotion predicted:  joy


In [ ]:
text3 = "I have a presentation tomorrow, im feeling nervous. I want it to go well, but i might fail."
print("Emotion predicted: ",predict(model2,T_vectorizer,text3))

Emotion predicted:  shame
